Weitere Schritte
- Doc-String dort wo es einen braucht einbauen, Code muss strukturiert sein. 
- ~~Client soll beendet werden können. In sendMessage noch ein if einbauen —> vielleicht mit einem Exit keyboard~~~
- ~~Auswählen an welche Person man noch schreiben möcht~~
- ~~URI Attribut anschauen? http Pfad? (eher unwichtig) (nice to have, keine prio~~~

In [1]:
%gui asyncio

from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import display
import asyncio
import os
import websockets
from re import search

def wait_for_change(widget):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget.on_click(getvalue, remove=True) 
    widget.on_click(getvalue)
    return future


class WebsocketClient:
    
    def __init__(self, server_ip, server_port=8080):
        """
        Init of Websocket_Client
        server_ip str: Either and IPv4 address or a name which can get resolved to an IP address.
        server_port int: default port is 80
        """
        self.server_ip = server_ip
        self.server_port = server_port
        self.user_name = os.path.expandvars("$USER")
        self.chat_input = widgets.Text(placeholder='Send your message like this --> receiver:message',description='Chat Input:',layout=Layout(width='80%'))
        self.chat_area = widgets.Textarea(value = '', disabled=True, description='Chat:', layout=Layout(width='99%', height='200px'))
        self.chat_button = widgets.Button(description='Send!',layout=Layout(width='19%'))
    
    async def connection(self):
        """
        Connects the client to the server
        """
        self.websocket = await websockets.connect(f"ws://{self.server_ip}:{self.server_port}")
        # The first message is the user name
        self.chat_area.value += "Connection was successful for client at " + f"ws://{self.server_ip}:{self.server_port}\n"
        await self.websocket.send(os.path.expandvars("$USER"))
        self.chat_area.value += f"{self.user_name} added to the chat\n"
        asyncio.create_task(self.send_message())
        asyncio.create_task(self.receive_message())
        display(VBox([HBox([self.chat_input, self.chat_button]), self.chat_area]))
        #await self.chatbot()

    ####################CHATBOT#######################   
    async def chatbot(self):
        self.websocket = await websockets.connect(f"ws://{self.server_ip}:{self.server_port}")
        await self.websocket.send("bot") #bot anmelden
        self.chat_area.value += f"bot added to the chat"
        while True: 
            response = await self.websocket.recv()
            sender, msg = response.split(':')
            await self.websocket.send({response}) # sender:bottext [::-1]

    async def send_message(self):
        """
        Sends messages
        """
        while True:
            await wait_for_change(self.chat_button)
            recv, msg = self.chat_input.value.split(':')
            self.chat_area.value += f"{self.user_name}:{msg}\n"
            if search('exit', self.chat_input.value):
                await self.websocket.close()
                self.chat_area.value += "Connection was successfully closed."
            else:
                await self.websocket.send(self.chat_input.value)
                self.chat_input.value = ""

    async def receive_message(self):
        """
        Receives messages
        """
        while True:
            response = await self.websocket.recv()
            self.chat_area.value += f"{response}\n"

In [2]:
#client = WebsocketClient(server_ip='michelle.ngu')
client = WebsocketClient(server_ip='julia.mueller')
await client.connection()

julia.mueller:michelle.ngu:hallo
julia.mueller:*:hallo

In [3]:
%gui asyncio

from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import display
import asyncio
import os
import websockets
from re import search

def wait_for_change(widget):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget.on_click(getvalue, remove=True) 
    widget.on_click(getvalue)
    return future


class WebsocketClient:
    
    def __init__(self, server_ip, server_port=8080):
        """
        Init of Websocket_Client
        server_ip str: Either and IPv4 address or a name which can get resolved to an IP address.
        server_port int: default port is 80
        """
        self.server_ip = server_ip
        self.server_port = server_port
        self.user_name = os.path.expandvars("$USER")
        self.chat_input = widgets.Text(placeholder='Send your message like this --> receiver:message',description='Chat Input:',layout=Layout(width='80%'))
        self.chat_area = widgets.Textarea(value = '', disabled=True, description='Chat:', layout=Layout(width='99%', height='200px'))
        self.chat_button = widgets.Button(description='Send!',layout=Layout(width='19%'))
    
    async def connection(self):
        """
        Connects the client to the server
        """
        self.websocket = await websockets.connect(f"ws://{self.server_ip}:{self.server_port}")
        # The first message is the user name
        self.chat_area.value += "Connection was successful for client at " + f"ws://{self.server_ip}:{self.server_port}\n"
        await self.websocket.send(os.path.expandvars("$USER"))
        self.chat_area.value += f"{self.user_name} added to the chat\n"
        asyncio.create_task(self.send_message())
        asyncio.create_task(self.receive_message())
        display(VBox([HBox([self.chat_input, self.chat_button]), self.chat_area]))
        #await self.chatbot()

    ####################CHATBOT#######################   
    async def chat_bot(self):
        self.websocket = await websockets.connect(f"ws://{self.server_ip}:{self.server_port}")
        await self.websocket.send("bot") #bot anmelden
        self.chat_area.value += f"bot added to the chat"
        while True: 
            response = await self.websocket.recv()
            sender, msg = response.split(':')
            await self.websocket.send({response}) # sender:bottext [::-1]

    async def send_message(self):
        """
        Sends messages
        """
        while True:
            await wait_for_change(self.chat_button)
            self.chat_area.value += f"An {self.chat_input.value}\n"
            if search('exit', self.chat_input.value):
                await self.websocket.close()
                self.chat_area.value += "Connection was successfully closed."
            else:
                await self.websocket.send(self.chat_input.value)
                self.chat_input.value = ""

    async def receive_message(self):
        """
        Receives messages
        """
        while True:
            response = await self.websocket.recv()
            self.chat_area.value += f"Von {response}\n"